In [2]:
# Google Colaboratory 사용 시
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [14]:
!ls gdrive/My\ Drive/02.개인/010.강의/'20200509-20200522 - assist 추천 시스템 강의'/20200515/data/tmdb_5000_movies.csv

'gdrive/My Drive/02.개인/010.강의/20200509-20200522 - assist 추천 시스템 강의/20200515/data/tmdb_5000_movies.csv'


In [15]:
!head gdrive/My\ Drive/02.개인/010.강의/'20200509-20200522 - assist 추천 시스템 강의'/20200515/data/tmdb_5000_movies.csv

budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {""id"": 878, ""name"": ""Science Fiction""}]",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""space war""}, {""id"": 3388, ""name"": ""space colony""}, {""id"": 3679, ""name"": ""society""}, {""id"": 3801, ""name"": ""space travel""}, {""id"": 9685, ""name"": ""futuristic""}, {""id"": 9840, ""name"": ""romance""}, {""id"": 9882, ""name"": ""space""}, {""id"": 9951, ""name"": ""alien""}, {""id"": 10148, ""name"": ""tribe""}, {""id"": 10158, ""name"": ""alien planet""}, {""id"": 10987, ""name"": ""cgi""}, {""id"": 11399, ""name"": ""marine""}, {""id"":

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [0]:
data = pd.read_csv('gdrive/My Drive/02.개인/010.강의/20200509-20200522 - assist 추천 시스템 강의/20200515/data/tmdb_5000_movies.csv')

In [104]:
data.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [105]:
data.shape

(4803, 20)

In [106]:
list(data)

['budget',
 'genres',
 'homepage',
 'id',
 'keywords',
 'original_language',
 'original_title',
 'overview',
 'popularity',
 'production_companies',
 'production_countries',
 'release_date',
 'revenue',
 'runtime',
 'spoken_languages',
 'status',
 'tagline',
 'title',
 'vote_average',
 'vote_count']

필요한 컬럼만 변수로 재정의

In [0]:
data = data[['id','genres', 'vote_average', 'vote_count','popularity','title',  'keywords', 'overview']]

투표 기준으로 상위 10% 영화만 추천하도록 필터링

In [108]:
# 투표 상위 10%가 되기 위해서 필요한 최소 투표수 확인
tmp_m = data['vote_count'].quantile(0.89)
tmp_m

1683.8999999999987

투표수가 1683건 이상인 영화만 필터링

In [0]:
m = data['vote_count'].quantile(0.9)
data = data.loc[data['vote_count'] >= m]

481의 영화만 남음

In [110]:
data.shape

(481, 8)

In [111]:
data[['genres', 'keywords']].head(2)

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na..."


literal_eval을 이용해 문자열 값을 list와 dictionary 데이터로 변경

In [0]:
data['genres'] = data['genres'].apply(literal_eval)
data['keywords'] = data['keywords'].apply(literal_eval)

In [113]:
data[['genres', 'keywords']].head(2)

,genres,keywords
0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...","[{'id': 1463, 'name': 'culture clash'}, {'id':..."
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'id': 270, 'name': 'ocean'}, {'id': 726, 'na..."


genres와 keywords에 있는 값들을 모두 추출하여 공백으로 연결

In [0]:
data['genres'] = data['genres'].apply(lambda x : [d['name'] for d in x]).apply(lambda x : " ".join(x))
data['keywords'] = data['keywords'].apply(lambda x : [d['name'] for d in x]).apply(lambda x : " ".join(x))

In [115]:
data.head(2)

,id,genres,vote_average,vote_count,popularity,title,keywords,overview
0,19995,Action Adventure Fantasy Science Fiction,7.2,11800,150.437577,Avatar,culture clash future space war space colony so...,"In the 22nd century, a paraplegic Marine is di..."
1,285,Adventure Fantasy Action,6.9,4500,139.082615,Pirates of the Caribbean: At World's End,ocean drug abuse exotic island east india trad...,"Captain Barbossa, long believed to be dead, ha..."


**콘텐츠 기반 필터링 추천(Content based filtering)**  
장르 데이터를 이용하여 콘텐츠 기반 추천을 수행

In [116]:
data.genres.head(2)

0    Action Adventure Fantasy Science Fiction
1                    Adventure Fantasy Action
Name: genres, dtype: object

문자열로 된 장르 데이터를 벡터로 변경

In [0]:
count_vector = CountVectorizer(ngram_range=(1,3))

In [0]:
c_vector_genres = count_vector.fit_transform(data['genres'])

In [119]:
c_vector_genres.shape

(481, 364)

In [122]:
c_vector_genres.toarray()

array([[1, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [0]:
v_vector_genres = count_vector.fit(data['genres'])

In [142]:
v_vector_genres.vocabulary_

{'action': 0,
 'action adventure': 1,
 'action adventure animation': 2,
 'action adventure comedy': 3,
 'action adventure crime': 4,
 'action adventure fantasy': 5,
 'action adventure science': 6,
 'action adventure thriller': 7,
 'action adventure war': 8,
 'action adventure western': 9,
 'action comedy': 10,
 'action comedy crime': 11,
 'action comedy drama': 12,
 'action comedy family': 13,
 'action comedy science': 14,
 'action crime': 15,
 'action crime drama': 16,
 'action crime mystery': 17,
 'action crime thriller': 18,
 'action drama': 19,
 'action drama adventure': 20,
 'action drama fantasy': 21,
 'action drama history': 22,
 'action drama horror': 23,
 'action drama mystery': 24,
 'action drama science': 25,
 'action drama thriller': 26,
 'action family': 27,
 'action family fantasy': 28,
 'action family romance': 29,
 'action fantasy': 30,
 'action fantasy comedy': 31,
 'action horror': 32,
 'action mystery': 33,
 'action mystery science': 34,
 'action mystery thriller': 3

In [0]:
#코사인 유사도를 구한 벡터를 미리 저장
gerne_c_sim = cosine_similarity(c_vector_genres, c_vector_genres)

In [127]:
gerne_c_sim

array([[1.        , 0.47140452, 0.35355339, ..., 0.        , 0.        ,
        0.        ],
       [0.47140452, 1.        , 0.33333333, ..., 0.        , 0.        ,
        0.        ],
       [0.35355339, 0.33333333, 1.        , ..., 0.        , 0.23570226,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.23570226, ..., 0.        , 1.        ,
        0.57735027],
       [0.        , 0.        , 0.        , ..., 0.        , 0.57735027,
        1.        ]])

In [128]:
gerne_c_sim[0]

array([1.        , 0.47140452, 0.35355339, 0.09622504, 0.57735027,
       0.47140452, 0.        , 0.57735027, 0.35355339, 0.70710678,
       0.19245009, 0.47140452, 0.35355339, 1.        , 0.57735027,
       0.35355339, 0.38490018, 0.70710678, 0.70710678, 0.5       ,
       0.23570226, 0.        , 0.48112522, 0.5       , 0.5       ,
       0.35355339, 0.70710678, 0.57735027, 0.23570226, 0.41666667,
       0.        , 0.57735027, 0.57735027, 0.23570226, 0.70710678,
       0.48112522, 0.09622504, 0.41666667, 0.        , 0.38490018,
       0.28867513, 0.2981424 , 1.        , 0.57735027, 0.23570226,
       0.        , 0.38490018, 0.48112522, 0.48112522, 0.33333333,
       0.2236068 , 0.57735027, 0.        , 0.57735027, 0.58333333,
       0.5       , 0.23570226, 0.5       , 0.09622504, 0.09622504,
       0.48112522, 0.11785113, 0.67082039, 0.47140452, 0.41666667,
       0.        , 0.19245009, 0.48112522, 0.35355339, 0.25      ,
       0.33333333, 0.57735027, 0.25      , 0.33333333, 0.16666

유사도가 큰 것 부터 정렬

In [0]:
#코사인 유사도를 구한 벡터를 미리 저장
gerne_c_sim = gerne_c_sim.argsort()[:, ::-1]

In [130]:
gerne_c_sim.shape

(481, 481)

In [131]:
gerne_c_sim

array([[  0,  13,  42, ..., 298, 297, 240],
       [ 11,   1, 200, ..., 329, 330, 240],
       [  2, 376, 216, ..., 314, 304, 240],
       ...,
       [478, 187,  12, ..., 326, 327,   0],
       [479, 466, 383, ..., 220, 224,   0],
       [480, 468, 294, ..., 246, 248,   0]])

In [0]:
def get_recommend_movie_list(df, movie_title, top=30):
    # 특정 영화와 비슷한 영화를 추천해야 하기 때문에 '특정 영화' 정보를 뽑아낸다.
    target_movie_index = df[df['title'] == movie_title].index.values
    
    #코사인 유사도 중 비슷한 코사인 유사도를 가진 정보를 뽑아낸다.
    sim_index = gerne_c_sim[target_movie_index, :top].reshape(-1)
    #본인을 제외
    sim_index = sim_index[sim_index != target_movie_index]

    #data frame으로 만들고 vote_count으로 정렬한 뒤 return
    result = df.iloc[sim_index].sort_values('vote_average', ascending=False)[:top]
    return result

In [143]:
get_recommend_movie_list(data, movie_title='The Dark Knight Rises',top=10)

,id,genres,vote_average,vote_count,popularity,title,keywords,overview
2091,274,Crime Drama Thriller,8.1,4443,18.174804,The Silence of the Lambs,based on novel psychopath horror suspense seri...,"FBI trainee, Clarice Starling ventures into a ..."
2760,264644,Drama Thriller,8.1,2757,66.113340,Room,based on novel carpet isolation kidnapping imp...,Jack is a young boy of 5 years old who has liv...
1850,111,Action Crime Drama Thriller,8.0,2948,70.105981,Scarface,miami corruption capitalism cuba prohibition b...,After getting a green card in exchange for ass...
1829,6977,Crime Drama Thriller,7.7,3003,53.645267,No Country for Old Men,texas drug traffic hitman united states–mexico...,"Llewelyn Moss stumbles upon dead bodies, $2 mi..."
3234,242582,Crime Drama Thriller,7.6,3395,89.095538,Nightcrawler,journalism underground tv station sociopath ho...,"When Lou Bloom, desperate for work, muscles in..."
119,272,Action Crime Drama,7.5,7359,115.040024,Batman Begins,himalaya martial arts dc comics crime fighter ...,"Driven by tragedy, billionaire Bruce Wayne ded..."
2730,44214,Drama Thriller,7.3,4430,91.285683,Black Swan,dancing competition sex dancer obsession paran...,"A ballet dancer wins the lead in ""Swan Lake"" a..."
1596,273481,Action Crime Drama Mystery Thriller,7.2,2416,55.424027,Sicario,mexico cia smoking texas fbi murder dirty cop ...,A young female FBI agent joins a secret CIA op...
717,75780,Crime Drama Thriller,6.3,2998,56.868399,Jack Reacher,based on novel sniper investigation police quarry,"In an innocent heartland city, five are shot d..."


In [136]:
data[data['title'] == 'The Dark Knight Rises']

,id,genres,vote_average,vote_count,popularity,title,keywords,overview
3,49026,Action Crime Drama Thriller,7.6,9106,112.31295,The Dark Knight Rises,dc comics crime fighter terrorist secret ident...,Following the death of District Attorney Harve...
